## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is, essentially, the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

* Describe RDDs as fundamental storage units in the Spark computing environment
* Create RDDs from Python collections
* Set number of partitions for parallelizing RDDs
* Review an RDD's dependency graph at different stages of processing. 
* Apply the map(func) transformation to a given function on all elements of an RDD in different partitions
* Use collect() action to trigger the processing stage of spark's lazy evaluation
* Use count() action to calculate the number of elements of a parallelized RDD
* Use filter(func) to filter unwanted data from RDDs
* Develop an understanding of Python's lambda functions for RDDs processing


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies will performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified.
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the Database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new data set from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base data set (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the Driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some of key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [2]:
data = range(1, 1001)
len(data)

#1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use 'local[*]' as the master.

In [3]:
import pyspark

sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `parallelize` method to create an rdd will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [7]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the getNumPartitions method.

In [8]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first n items in the RDD
* `top`: returns the top n items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [9]:
# first
rdd.first()

1

In [11]:
# take
rdd.take(7)

[1, 2, 3, 4, 5, 6, 7]

In [12]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [13]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 be a random number from 0-1.

In [14]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([1.34077141e-01, 1.73450375e+00, 1.48287220e+00, 3.94015518e+00,
       1.15686892e+00, 1.85508515e+00, 1.27391598e+00, 1.70037269e+00,
       4.23139174e+00, 2.70861767e+00, 8.23654459e-01, 7.85700469e+00,
       1.26927156e+01, 1.18251970e+01, 5.47291861e+00, 2.00440274e+00,
       5.17109155e+00, 6.01204723e-01, 1.84495045e+01, 8.20462607e+00,
       1.64824641e+00, 8.30099664e-01, 1.32533921e+01, 5.32788946e+00,
       1.07707977e+01, 1.23031407e+01, 1.74433751e+01, 1.96885697e+01,
       9.35246368e+00, 2.42044124e+01, 1.87052103e+01, 2.82376397e+01,
       2.74931689e+01, 3.27191619e+01, 3.11679404e+01, 2.34236595e+01,
       1.55302500e+01, 1.64620173e+01, 1.84835618e+01, 1.57850980e+01,
       3.71289833e+00, 1.94459502e+01, 3.19961091e+01, 3.06317910e+01,
       9.33484345e+00, 2.31549772e+01, 4.18796349e+01, 1.15944059e+01,
       3.29745293e+00, 4.71482666e+01, 6.75184495e+00, 1.05615764e+01,
       2.27099550e+00, 4.74842682e+01, 4.93765481e+01, 4.56737394e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [15]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.top(10)

[937.0425115884616,
 936.0531223867065,
 923.0549546309142,
 918.6753836744024,
 916.6709555502418,
 896.9601939319504,
 887.3689718725991,
 886.9532634065077,
 871.0553306101484,
 868.024244058231]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax` that returns the amount of money our company will receive after the sale tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the map method and assign it to a variable `renenue_minus_tax`

In [16]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [17]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.12335096974677431,
 1.595743447671913,
 1.3642424258027905,
 3.6249427664681204,
 1.0643194091438783,
 1.7066783334081834,
 1.1720026970076578,
 1.5643428755392694,
 3.892880404190706,
 2.4919282584541675]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a map method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [18]:
discounted = revenue_minus_tax.map(lambda x : x * 0.9)

In [19]:
discounted.take(10)

[0.11101587277209689,
 1.4361691029047217,
 1.2278181832225115,
 3.2624484898213084,
 0.9578874682294906,
 1.5360105000673652,
 1.054802427306892,
 1.4079085879853426,
 3.5035923637716357,
 2.2427354326087507]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff use a collection method to see the 15 costliest items.

In [20]:
price_items.map(sales_tax).map(lambda x : x * 0.9).top(15)

[775.8711995952463,
 775.0519853361931,
 764.289502434397,
 760.6632176824053,
 759.0035511956004,
 742.6830405756549,
 734.7415087105121,
 734.3973021005885,
 721.233813745203,
 718.7240740802154,
 716.9916750774636,
 714.7290652327722,
 714.6744401468424,
 712.3816064907228,
 711.9168195087047]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [22]:
discounted.toDebugString()

b'(10) PythonRDD[12] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use flatMap rather than a simple map. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will a tuple with (original price, post-discount price).

In [23]:
mapped = price_items.map(lambda x: (x, x * 0.92 * 0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.1340771410291025, 0.11101587277209689), (1.7345037474694707, 1.4361691029047217), (1.4828722019595548, 1.2278181832225115), (3.940155180943609, 3.2624484898213084), (1.1568689229824765, 0.9578874682294906), (1.855085145008895, 1.5360105000673652), (1.2739159750083235, 1.054802427306892), (1.7003726908035537, 1.4079085879853426), (4.23139174368555, 3.5035923637716357), (2.7086176722327906, 2.2427354326087507)]


Note that we have 1000 tuples created to our specification. Let's take a look at how flatMap differs in its implementation. Use the `flatMap` method with the same function you created above.

In [24]:
flat_mapped = price_items.flatMap(lambda x : (x, x * 0.92 * 0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.1340771410291025, 0.11101587277209689, 1.7345037474694707, 1.4361691029047217, 1.4828722019595548, 1.2278181832225115, 3.940155180943609, 3.2624484898213084, 1.1568689229824765, 0.9578874682294906]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use flatmap rather than map in order to properly reduce to our specifications. This is not one of those instances, but int he upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the filter method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a filter function to meet the consultant's suggestion's specifications. set RDD = `selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [25]:
# use the filter function
selected_items = discounted.filter(lambda x: x > 300)

# calculate total remaining in inventory 
selected_items.count()

280

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use a reduce method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [26]:
selected_items.reduce(lambda x, y: x + y)

130555.59946912373

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [27]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(50, 325.93952563922926),
 (21, 365.0901861127488),
 (14, 327.16425368907124),
 (13, 362.33637514131107),
 (9, 336.2632218692111),
 (17, 346.3200441913809),
 (6, 337.5514182690946)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called __reduceByKey__ to perform reducing operations while grouping by keys. After you have calculated the total, use the __sortBy__ method on the RDD to rank the users from the highest spending to the least spending.



In [28]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(10)

[(10, 4676.211172914879),
 (40, 5336.409484180464),
 (30, 2923.9240717738585),
 (20, 2860.979273814035),
 (50, 2282.1929577895116),
 (31, 3090.175979006046),
 (21, 3625.1910510239377),
 (1, 2317.007424179561),
 (11, 2809.9243336944182),
 (41, 1556.5996039554652)]

In [29]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1], ascending=False).collect()

[(40, 5336.409484180464),
 (37, 5128.45168087337),
 (10, 4676.211172914879),
 (16, 4386.15415435127),
 (7, 4282.613593388345),
 (32, 4077.904517186972),
 (12, 3873.011797206666),
 (3, 3867.998800142999),
 (21, 3625.1910510239377),
 (49, 3537.971449702578),
 (35, 3375.4176523436595),
 (26, 3354.137123360688),
 (46, 3336.825986987219),
 (18, 3307.9037763474944),
 (38, 3188.004643977991),
 (31, 3090.175979006046),
 (28, 3072.4613563556177),
 (6, 3071.1188615144197),
 (34, 3033.494419925007),
 (30, 2923.9240717738585),
 (20, 2860.979273814035),
 (11, 2809.9243336944182),
 (4, 2808.8762299776063),
 (13, 2617.5108339201606),
 (27, 2608.65073235389),
 (39, 2509.524073529467),
 (9, 2485.0968522461008),
 (23, 2427.582923226258),
 (1, 2317.007424179561),
 (25, 2312.659748094169),
 (50, 2282.1929577895116),
 (44, 2181.2111051394013),
 (14, 1926.864841937421),
 (24, 1895.1192895201557),
 (45, 1819.9741815480722),
 (36, 1705.5409897475943),
 (15, 1687.4004101233863),
 (48, 1667.3067590470473),
 (29

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [30]:
total_items = sales_data.countByKey()
sorted(total_items.items(), key=lambda x: x[1], reverse=True)

[(5, 11),
 (43, 11),
 (21, 11),
 (48, 9),
 (19, 9),
 (36, 9),
 (37, 8),
 (34, 8),
 (7, 8),
 (4, 8),
 (47, 8),
 (16, 8),
 (22, 7),
 (35, 7),
 (15, 7),
 (40, 7),
 (29, 7),
 (28, 7),
 (13, 7),
 (45, 6),
 (20, 6),
 (2, 6),
 (23, 6),
 (24, 6),
 (46, 5),
 (10, 5),
 (49, 5),
 (11, 5),
 (27, 5),
 (39, 5),
 (12, 5),
 (50, 4),
 (14, 4),
 (6, 4),
 (17, 4),
 (30, 4),
 (42, 4),
 (26, 4),
 (41, 4),
 (32, 4),
 (3, 3),
 (44, 3),
 (1, 3),
 (31, 3),
 (33, 3),
 (8, 2),
 (18, 2),
 (9, 2),
 (38, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD, and extracting lineage and of an RDD in a spark application. We also used transformations and actions to perform calculations across RDDs on a distributed setup. Up next, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
